## Data Preparation

In [1]:
import nltk
import json
import math
import numpy as np  
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
def loadData():
    for line in open('Sarcasm_Headlines_Dataset.json','r'):
        yield eval(line)

In [3]:
Data=list(loadData())

In [4]:
print(Data[0])
print(Data[0]['headline'])
print(Data[0]['is_sarcastic'])    

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}
former versace store clerk sues over secret 'black code' for minority shoppers
0


In [5]:
X=[]
Y=[]

In [6]:
for i in range(len(Data)):
    X.append(Data[i]['headline'])
    Y.append(Data[i]['is_sarcastic'])    

In [7]:
print(X[2])
print(Y[2])
size=len(X)
#size=15000

mom starting to fear son's web series closest thing she will have to grandchild
1


## Tokenization Over All

In [8]:
#Tokens
wordsTORemove=[]
tokens=[]
for i in range(size):
    temp=nltk.word_tokenize(X[i])
    for j in range(len(temp)):
        tokens.append(temp[j])

In [9]:
print(len(tokens))

158227


In [10]:
#Decapitalized
tokens=[element.lower() for element in tokens]

In [11]:
#Remove Special characters
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~0123456789")
tokens=[x.translate(removetable) for x in tokens]

In [12]:
#Importing Stoplist
stopWord=open("Stopword-List.txt").read()
stopWord=nltk.word_tokenize(stopWord)
# Removing StopWords
tokens=[x for x in tokens if x.isalnum() and x not in stopWord]

In [13]:
#word Frequency
fdist=nltk.FreqDist(tokens)

In [14]:
# Unique Tokens 
tokens=list(set(tokens))
print(len(tokens))

20216


In [15]:
#Counting Frequency
#Removing words freq <=3
for words in tokens:
    if(fdist[words]<=3):
        tokens.remove(words)
        wordsTORemove.append(words)

In [16]:
len(tokens)

11629

In [17]:
#Removing Token Tags
tagg=nltk.tag.pos_tag(tokens)
taggs=set()
for word,type1 in tagg:
    if(type1=='NNP' or type1=='FW' or type1=='PRP'):
        taggs.add(word)    

In [18]:
tokens=list(set(tokens)-taggs)

In [19]:
#Removing words with length 2
for words in tokens:
    if len(words)<=2:
        tokens.remove(words)
        wordsTORemove.append(words)

In [20]:
len(tokens)

11471

## Document Wise Tokenization 

In [21]:
#Tokens
docToken=[]
for i in range(size):
    docToken.append(nltk.word_tokenize(X[i]))    

In [22]:
print(docToken[106])

['l.a.', 'grants', 'clippers', '$', '12', 'for', 'new', 'nets']


In [23]:
#Decaptilized Doc Wise
for x in range(size):
    docToken[x]=[element.lower() for element in docToken[x]]

In [24]:
#Remove Special characters Doc Wise
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
for x in range(size):
    docToken[x]=[y.translate(removetable) for y in docToken[x]]

In [25]:
#removing StopWords
for x in range(size):
    docToken[x]=[y for y in docToken[x] if y.isalnum() and y not in stopWord]

In [26]:
#Removing Token Tags
for x in range(size):
    docToken[x]=list(set(docToken[x])-taggs)

In [27]:
# Counting Frequency
# Removing words freq <=3
for x in range(size):
    for words in docToken[x]:
        if len(words)<=2:
            docToken[x].remove(words)
        elif words in wordsTORemove:
            docToken[x].remove(words)

In [28]:
print(docToken[104])

['top', '2015', 'italy']


## BIGRAM

In [29]:
#bigram=[]
#for i in range(50):
 #   ls=list(nltk.bigrams(docToken[i]))
  #  for j in ls:
   #     bigram.append(list(j))

In [30]:
#len(bigram)

## TF-idf

In [31]:
docV={}
for x in range(size):
    docV[x]=dict.fromkeys(tokens,0) 
    
#Term Frequency in  a document
for x in range(size):
    for word in docToken[x]:
        try:
            docV[x][word]+=1
        except KeyError:
            pass

In [32]:
# Unconnent karna
#print(docV[106]['grants'])

In [33]:
#tf
tfDocV={}
for x in range(size):
    tfDocV[x]={}
    for word,count in docV[x].items():
        tfDocV[x][word]=count

In [34]:
print(len(tfDocV[1]))

11471


In [35]:
#unique Token Doc wise
for x in range(size):
    docToken[x]=set(docToken[x])
    docToken[x]=list(set(docToken[x]))

In [36]:
wordDcount=dict.fromkeys(tokens,0)

for word in tokens:
    for x in range(size):
        if word in docToken[x]:
            wordDcount[word]+=1

In [37]:
len(wordDcount)

11471

In [38]:
#idf            
idfDict = {}
for word in tokens:
    if wordDcount[word]>0:
        count=wordDcount[word]
        if count>size:
            count=size
    
    if count==0:
        count=1
        
    idfDict[word]=math.log(size/count)        

In [39]:
len(idfDict)

11471

In [40]:
#tf-idf    
tfidf={}
for x in range(size):
    tfidf[x]={}
    for word in docV[x]:
        tfidf[x][word]=tfDocV[x][word]*idfDict[word]

In [41]:
print(tfidf[106]['grants'])

8.006367567650246


In [42]:
#bigram={}
#for i in range(size):
#    ls=[]
#    for j in range(len(docToken[i])-1):
#        for k in range(j,len(docToken[i])-1):
#            ls.append([docToken[i][j],docToken[i][k+1]])
        
#    bigram[i]=ls

In [43]:
#bigram[104]

In [44]:
#bigramVal={}
#for i in range(size):
#    bigramVal[i]={}
#    for x in range(len(bigram[i])):  
#        try:
#            bigramVal[i][bigram[i][x][0]+" "+bigram[i][x][1]]=tfidf[i][bigram[i][x][0]]+tfidf[i][bigram[i][x][1]]
#        except KeyError:
#            pass

In [45]:
#Max3=[]
#for x in range(size):
#    Max3.append(sorted(bigramVal[x],key=bigramVal[x].get, reverse=True)[:3])

In [46]:
#Max3[104]

In [47]:
#len(tfidf)

In [48]:
df=pd.DataFrame(tfidf)
df=np.transpose(df)

In [65]:
df.to_csv('pData.csv')

In [ ]:
#df_csv = pd.read_csv('pData')

In [63]:
scaler=StandardScaler()
#scaler=MinMaxScaler()
scaler.fit(df)
scaled_data=scaler.transform(df)

MemoryError: 

In [50]:
scaled_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
pca=PCA(n_components=0.92)
pca.fit(scaled_data)
x_pca=pca.transform(scaled_data)

MemoryError: 

In [ ]:
print(scaled_data.shape)
print(x_pca.shape)

In [ ]:
#kmeans = KMeans(n_clusters=3,random_state=0).fit(x_pca)

In [ ]:
#for x in range(size):
#    if kmeans.labels_[x]==2:
#        print(x," ",kmeans.labels_[x])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, Y[:size], test_size=0.20,random_state=42)

In [ ]:
clf = svm.SVC()
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
##Test
#pca=PCA(n_components=0.92, svd_solver='full')
#pca.fit(scaled_data)
#x_pca=pca.transform(scaled_data)
#print(sum(pca.explained_variance_ratio_))
#print(shape(x_pca))

In [ ]:
#print(x_pca.shape)

In [ ]:
#plt.figure(figsize=(10,10))
#plt.scatter(x_pca[:,0],x_pca[:,1])
#plt.xlabel('First principle component')
#plt.ylabel('Second principle component')